In [1]:
import numpy as np
import pandas as pd

In [2]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})

In [3]:
pd.concat([df1, df2])

,employee,group,hire_date
0,Bob,Accounting,NaN
1,Jake,Engineering,NaN
2,Lisa,Engineering,NaN
3,Sue,HR,NaN
0,Lisa,NaN,2004.0
1,Bob,NaN,2008.0
2,Jake,NaN,2012.0
3,Sue,NaN,2014.0


In [6]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [7]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
pd.merge(df3, df4)

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


In [8]:

df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
pd.merge(df3, df5)

,employee,group,hire_date,skills
0,Bob,Accounting,2008,math
1,Bob,Accounting,2008,spreadsheets
2,Jake,Engineering,2012,coding
3,Jake,Engineering,2012,linux
4,Lisa,Engineering,2004,coding
5,Lisa,Engineering,2004,linux
6,Sue,HR,2014,spreadsheets
7,Sue,HR,2014,organization


In [13]:

df6 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})

pd.merge(df3, df6, left_on='employee', right_on='name').drop('name', axis='columns')

,employee,group,hire_date,salary
0,Bob,Accounting,2008,70000
1,Jake,Engineering,2012,80000
2,Lisa,Engineering,2004,120000
3,Sue,HR,2014,90000


In [14]:
# Following are shell commands to download the data
#!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
#!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
#!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57935  100 57935    0     0   132k      0 --:--:-- --:--:-- --:--:--  132k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   835  100   835    0     0   3739      0 --:--:-- --:--:-- --:--:--  3744
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   872  100   872    0     0   3660      0 --:--:-- --:--:-- --:--:--  3648


In [58]:
state_abbrevs = pd.read_csv('data/state-abbrevs.csv')
state_areas = pd.read_csv('data/state-areas.csv')
state_population = pd.read_csv('data/state-population.csv')

In [59]:
print(state_abbrevs.head())
print(state_areas.head())
print(state_population.head())

        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0


In [60]:
# rank US states and territories by their 2010 population density
state_population = state_population[(state_population.year==2010) & (state_population.ages=='total')]
df = pd.merge(state_population, state_abbrevs, left_on='state/region', right_on='abbreviation', how='outer')
df.loc[df['state/region']=='PR', 'state'] = 'Puerto Rico'
df.loc[df['state/region']=='USA', 'state'] = 'United States'
df.drop('abbreviation', axis=1, inplace=True)

print(df.isnull().any())

df = pd.merge(df, state_areas, how='outer')
print(df.head())
print(df.isnull().any())
print(df[df['area (sq. mi)'].isnull()])
df.dropna(how='any', inplace=True)

state/region    False
ages            False
year            False
population      False
state           False
dtype: bool
  state/region   ages  year  population       state  area (sq. mi)
0           AL  total  2010   4785570.0     Alabama        52423.0
1           AK  total  2010    713868.0      Alaska       656425.0
2           AZ  total  2010   6408790.0     Arizona       114006.0
3           AR  total  2010   2922280.0    Arkansas        53182.0
4           CA  total  2010  37333601.0  California       163707.0
state/region     False
ages             False
year             False
population       False
state            False
area (sq. mi)     True
dtype: bool
   state/region   ages  year   population          state  area (sq. mi)
52          USA  total  2010  309326295.0  United States            NaN


In [61]:
df['pop_density'] = df['population'] / df['area (sq. mi)']
density = df.sort_values(by='pop_density', ascending=False)[['state', 'pop_density']]

In [62]:
density.head()

,state,pop_density
8,District of Columbia,8898.897059
51,Puerto Rico,1058.665149
30,New Jersey,1009.253268
39,Rhode Island,681.339159
6,Connecticut,645.600649


In [63]:
density.tail()

,state,pop_density
41,South Dakota,10.583512
34,North Dakota,9.537565
26,Montana,6.736171
50,Wyoming,5.768079
1,Alaska,1.087509


## Aggregation and Grouping

In [64]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [65]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [67]:
planets.isnull().sum()

method              0
number              0
orbital_period     43
mass              522
distance          227
year                0
dtype: int64

In [71]:
planets.describe()

,number,orbital_period,mass,distance,year
count,1035.000000,992.000000,513.000000,808.000000,1035.000000
mean,1.785507,2002.917596,2.638161,264.069282,2009.070531
std,1.240976,26014.728304,3.818617,733.116493,3.972567
min,1.000000,0.090706,0.003600,1.350000,1989.000000
25%,1.000000,5.442540,0.229000,32.560000,2007.000000
50%,1.000000,39.979500,1.260000,55.250000,2010.000000
75%,2.000000,526.005000,3.040000,178.500000,2012.000000
max,7.000000,730000.000000,25.000000,8500.000000,2014.000000


In [70]:
# count discovered planets by method
planets.groupby('method')['number'].count()

method
Astrometry                         2
Eclipse Timing Variations          9
Imaging                           38
Microlensing                      23
Orbital Brightness Modulation      3
Pulsar Timing                      5
Pulsation Timing Variations        1
Radial Velocity                  553
Transit                          397
Transit Timing Variations          4
Name: number, dtype: int64

In [76]:
# count discovered planets by method and by decade

decades = 10 * (planets['year'] // 10)
decades = decades.astype('str') + 's'
decades

0       2000s
1       2000s
2       2010s
3       2000s
4       2000s
        ...  
1030    2000s
1031    2000s
1032    2000s
1033    2000s
1034    2000s
Name: year, Length: 1035, dtype: object

In [79]:
planets.groupby(['method', decades])['number'].count().unstack().fillna(0)

year,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,3.0,6.0
Imaging,0.0,0.0,20.0,18.0
Microlensing,0.0,0.0,10.0,13.0
Orbital Brightness Modulation,0.0,0.0,0.0,3.0
Pulsar Timing,0.0,3.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,28.0,309.0,215.0
Transit,0.0,0.0,62.0,335.0
